In [15]:
import pandas as pd
import re 
import itertools
from difflib import SequenceMatcher

root = "Data/csvs/basal_ganglia/regions"
sim_csv_loc = "/region_similarity.csv"


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


## Prepare regions and regions_other csvs
df_all_regions = pd.read_csv(root + "/all_regions.csv", dtype="object")

df = pd.DataFrame(columns = ["ID1", "Region_name_1", "ID2", "Region_name_2", "Sim"])

# Put region names and ID into tuple list 
subset = df_all_regions[["ID", "Region_name"]]
region_name_tuples = [tuple(x) for x in subset.to_numpy()]

# Find all combinations of region_names and look at similarity in name
for a, b in itertools.combinations(region_name_tuples, 2):
    id1, reg1 = a
    id2, reg2 = b
    sim_score = similar(reg1, reg2)
    
    if(sim_score > 0.7):
        a_row = pd.Series([id1, reg1, id2, reg2, sim_score], index = ["ID1", "Region_name_1", "ID2", "Region_name_2", "Sim"])
        df = df.append(a_row, ignore_index=True)


# Store similarities
df_sorted = df.sort_values('Sim')
df_sorted.to_csv(root + sim_csv_loc, encoding='utf-8')

print("Similarities stored in", sim_csv_loc)

Similarities stored in /region_similarity.csv


In [23]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")



In [24]:
## Then find the regions that correspond to each other and stor that in a new CSV file

# Add relation to all areas that define positions
positioning = ["caudal", "rostral", "ventral", "dorsal"]
area_describing = ["internal", "compact", "core", "shell"]

df_sims = pd.read_csv(root + sim_csv_loc, converters = {"Sims": float})

# ALl with score above 0.95 are the same
    # Also the same: Substantia innominata, basal",103,"Substantia innominata, basal part" 0.91
    
df_equals = df_sims.loc[df_sims['Sim'] > 0.95]
df_sorted.to_csv(root + "/regions_equal.csv", encoding='utf-8')

from neo4j import GraphDatabase, basic_auth
import os

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth("neo4j", "Kaffe1234"))

# Relationship EQUALS between equal BrainRegion nodes
csv_file_path = "file:///%s" % get_csv_path("regions_equal.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion {id: row.ID1})
        MATCH (c:BrainRegion {id: row.ID2 })
        MERGE (a)-[:EQUALS]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("EQUALS", session)

## TODO add rel for belongs-to/part of 


Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2283)'")
Failed to write data to connection Address(host='localhost', port=7687) (Address(host='127.0.0.1', port=7687)); ("0; 'Underlying socket connection gone (_ssl.c:2283)'")


Added 7210 relationships of type EQUALS
